Aufgabe 1: [15/120 Punkte] Nenne 5 **verschiedene** Anwendungen wo Key-Value-Datenbanken Vorteile gegenüber Relationalen Datenbanken haben, und beschreibe spezifisch (aber dennoch stichwortartig) für jeden Fall, was genau der jeweilige Vorteil ist (und: was potentielle Nachteile sind)

Aufgabe 2: [15/120 Punkte] erkläre was denn nun wirklich die Hauptunterschiede zwischen einem Data Warehouse und einem Data Lake sind und gib Beispiele für Anwendungen in denen entweder ein Data Warehouse oder ein Data Lake die bessere Wahl ist

Aufgabe 3: [15/120 Punkte] im folgenden Code hat sich der Fehlerteufel eingeschlichen. Finde die 5 Fehler (10 Punkte), und beschreibe stichwortartig, was der Code denn tun würde, wenn keine Fehler drin wären. (5 Punkte)

In [ ]:
from bson.son import SON

# MongoClient Import fehlt, wird hier nachfolgend aufgerufen, wurde aber nie importiert
client = MongoClient("localhost", 27017)

# Die Syntax für die Connection zur Datenbank & Collection ist nicht korrekt, muss separat gemacht werden, und nicht in einem Schritt zusammen
connect = client["database"],["collection"]

# $mismatch existiert nicht, $match sollte verwendet werden
pipe2 = [ { '$mismatch' : { 'origin' : 'ATL',
                         'dest' : 'BOS',
                         'dayofweek' : 3
                       }
          },
          { '$group' : { '_id' : { 'origin' : '$origin',
                                   'destination' : '$dest'
                                 },
                         'Failure' : { '$sum' : { '$cond' : [{ '$eq' : ['$cancelled', 1]}, 1, 0 ]} },
                         'Success' : { '$sum' : { '$cond' : [{ '$eq' : ['$cancelled', 0]}, 1, 0 ]} },
                         'Total' : { '$sum' : 1 }
                        }
           },
           { '$project' : { 'Failure' : 1,
                            'Success' : 1,
                            'Total' : 1,
                            'FailPercent' : { '$divide' : [ '$Failure', '$Total' ] }
                          }
           },
           # im SON-Statement ist kein Fehler, das gehoert so 
           { '$sort' : SON([('_id.origin', 1), ('_id.destination', 1 )]) }
         
# Hier fehlt eine schließende Klammer ] für die Syntax der Pipeline, um dieser zu sagen dass die Pipeline nun zu Ende ist.
         
# Hier bei result = wird eine Variable "connection" aufgerufen, die vorher nicht definiert wurde, die Variable, die hier benutzt werden muss, ist "connect" wie oben definiert worden ist
result = connection.aggregate(pipeline=pipe2)
print(result)
         
         
# Wenn keine Fehler in dem Code drin wären, würde es zuerst eine Verbindung zu einer lokalen MongoDB (localhost) auf port 27017 herstellen, dann eine Verbindung zu der Datenbank "Database",
# und zur Collection "collection" aufgebaut. Danach wird eine Pipeline erstellt wo ein match, ein group, ein project und ein sort definiert werden, welche dann zum Schluss in unserer Collection abgerufen wird. Zuletzt wird dann noch das ergebnis geprinted.


SyntaxError: '[' was never closed (2221102690.py, line 10)

Aufgabe 4: [15/120 Punkte] Erkläre was lazy evaluation ist an Hand von 2 in den Vorlesungen behandelten Technologien/Ansätzen (1 Lazy, eine nicht lazy) - und erläutere, was die Auswirkungen in der Praxis sind.

Aufgabe 5: [40/120 Punkte]
    In Moodle findest Du Super-Markt Daten. Werte diese in Spark aus, und finde heraus:
    
    1) wie viele Einkäufe wurden von Frauen, bzw von Männern getätigt?
    2) wie viel Umsatz wurde an den 3 Standorten jeweils getätigt - gib diesen zusammen mit dem NAMEN des jeweiligen Standorts aus
    3) welche Bezahlart die höchste durchschnittliche User-Satisfaction hat
    
Exportiere die Ergebnisse von Teilaufgabe 3 als CSV-Datei - stelle dabei sicher, dass unabhängig von der Größe des Datensatzes nur 1 einzige CSV-Datei generiert wird.

Beschreibe für jede der Teilaufgaben wie Du die Daten dazu aufbereiten würdest (Stichwort: Designentscheidungen)  

In [5]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Name der DB") \
      .getOrCreate() 

In [4]:
spark

In [21]:
# Da das Dateiformat nicht gepasst hat, wurde die Datei in LibreOffice geöffnet, und im Anschluss als CSV Datei gespeichert, um hier eingelesen werden zu können

file = spark.read.csv('supermarket_sales.csv', header= True)
file.show(truncate = False)

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |13:08|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |f     |Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |10:29|Cash       |76.4  |4.761904762            |3.82        |9.6   |
|631-41-3108|A     |null     |Normal       |M

In [46]:
# 1) wie viele Einkäufe wurden von Frauen, bzw von Männern getätigt?

# pyspark.sql.functions importieren für regexp_replace Funktion

import pyspark.sql.functions as F

# Wir schauen erstmal, wie viele verschiedene "Arten" von GenderTypen es gibt

anzahlGender = file.groupBy('Gender').count()
anzahlGender.show()

# Es gibt jeweils für Männer und Frauen 3 Typen : m, male, Male / f, female, Female
# Jetzt könnte man die Werte in den Spalten über RegEx abänder, sodass nur noch m/f drin vorkommen würde

updategender1 = file.withColumn("Gender", F.regexp_replace(F.col("Gender"), "male", "m"))
updategender2 = updategender1.withColumn("Gender", F.regexp_replace(F.col("Gender"), "Male", "m"))
updategender3 = updategender2.withColumn("Gender", F.regexp_replace(F.col("Gender"), "fem", "f"))
updategender4 = updategender3.withColumn("Gender", F.regexp_replace(F.col("Gender"), "Fem", "f"))

anzahlGender = updategender4.groupBy('Gender').count()
anzahlGender.show()


+------+-----+
|Gender|count|
+------+-----+
|     m|    4|
|     f|    3|
|Female|  110|
|female|    1|
|  male|    1|
|  Male|  136|
+------+-----+

+------+-----+
|Gender|count|
+------+-----+
|     m|  141|
|     f|  114|
+------+-----+



In [75]:
# 2) wie viel Umsatz wurde an den 3 Standorten jeweils getätigt - gib diesen zusammen mit dem NAMEN des jeweiligen Standorts aus
# Nach City gruppieren und dazu noch die Summe aggregaten die als Umsatz getätigt wurden
aggregationpipeline = {'Total' : 'sum'}
standorte = file.groupBy('Branch').agg(aggregationpipeline)

updateStandort1 = standorte.withColumn("Branch", F.regexp_replace(F.col("Branch"), "A", "Yangon"))
updateStandort2 = updateStandort1.withColumn("Branch", F.regexp_replace(F.col("Branch"), "B", "Mandalay"))
updateStandort3 = updateStandort2.withColumn("Branch", F.regexp_replace(F.col("Branch"), "C", "Naypyitaw"))

updateStandort3.show(truncate = False)

+---------+--------------------+
|Branch   |sum(Total)          |
+---------+--------------------+
|Mandalay |2.02011719595E7     |
|Naypyitaw|1.6445864925000004E7|
|Yangon   |1.5382536666000003E7|
+---------+--------------------+



In [76]:
# 3) welche Bezahlart die höchste durchschnittliche User-Satisfaction hat

userSatisfaction = file.groupBy('Payment').count()
userSatisfaction.show()

# Erstmal die Bezahltypen anpassen damit wir ordentlich gruppieren können

updatepayment1 = file.withColumn("Payment", F.regexp_replace(F.col("Payment"), "creditcard", "Kreditkarte"))
updatepayment2 = updatepayment1.withColumn("Payment", F.regexp_replace(F.col("Payment"), "Creditcard", "Kreditkarte"))
updatepayment3 = updatepayment2.withColumn("Payment", F.regexp_replace(F.col("Payment"), "credit card", "Kreditkarte"))
updatepayment4 = updatepayment3.withColumn("Payment", F.regexp_replace(F.col("Payment"), "Credit card", "Kreditkarte"))
updatepayment5 = updatepayment4.withColumn("Payment", F.regexp_replace(F.col("Payment"), "Cash", "cash"))
updatepayment6 = updatepayment5.withColumn("Payment", F.regexp_replace(F.col("Payment"), "ewallet", "Ewallet"))

# Jetzt nach der Payment-Methode gruppieren und die durschnittliche Rating Bewertung aggregaten
payment = updatepayment6.groupBy('Payment').agg({'Rating' : 'avg'})

# Jetzt noch nach dem Rating sortieren
result = payment.orderBy('avg(Rating)')
result.show()

# Als eine CSV-Datei exportieren
result.coalesce(1).write.csv("resultdata.csv", header = True)


+-----------+-----+
|    Payment|count|
+-----------+-----+
| creditcard|    1|
|       cash|    2|
| Creditcard|    3|
|    ewallet|    2|
|    Ewallet|   88|
|       Cash|   82|
|credit card|    1|
|Credit card|   76|
+-----------+-----+

+-----------+-----------------+
|    Payment|      avg(Rating)|
+-----------+-----------------+
|    Ewallet|6.795555555555555|
|       cash|6.983333333333333|
|Kreditkarte| 7.13333333333333|
+-----------+-----------------+



Aufgabe 6: [20/60 Punkte] Ein regionales Unternehmen hat ein großes Investment durch eine Venture Capital Firma bekommen, und möchte jetzt international expandieren. 

Euer Auftrag ist es das Unternehmen zum Thema "Datenbanken" (die für die internationale Expansion benötig werden) zu beraten.

Das bisherige Hauptprodukt des Unternehmens sind mit Schokolade überzogene Fruchtgummis. Im Zuge der Expansion möchte das Unternehmen aber auch gleichzeitig neue Produkte einführen, bzw an regionale Geschmäcker anpassen. Dadurch, dass die Expansion überall gleichzeitig erfolgen soll, sind am Anfang wenig Möglichkeiten direkt Kundenumfragen zu machen - statt dessen soll die Resonanz auf Marketingcampagnen und die Interaktion mit der jeweiligen Landeswebseite genutzt werden, um lokale "Geschmäcker" und damit das Potential für weitere Produkte zu erforschen.

Der bisher einzige Vertriebskanal ist eine Online-Präsenz, zumindest anfänglich soll dies auch so bleiben, jedoch mit potentiell separaten Webseiten für verschiedene Länder/Regionen. Hierzu besteht aber noch keine fest definierte Strategie, der Unternehmer würde von Euch auch gerne wissen, was für Möglichkeiten, Vor- und Nachteile, und Risiken verschiedener Ansätze (aus Sicht der Nutzung von Datenbanken) sind.

Erläutere **stichwortartig**:

1) welche Technologien in welchem Bereich eingesetzt werden, und warum
2) was für Strategien, bzw Tools Ihr einsetzen wollt um 
    - die schon vorhandenen Daten zu migrieren
    - neue Daten mit den alten zu verbinden
    - benötigte Daten zu erhalten
3) welche Strategien sich anbieten für das Reporting, die Datenhaltung, Backups etc.
4) was die Auswirkungen verschiedener Expansionsstrategien (z.B. Marketing, Website etc) auf die Nutzung/Nützlichkeit/Performance der Datenbank(en) wäre 
5) Ob es "Bottlenecks" oder mit (hohem) Risiko verbundene Datenbank-Komponenten gibt, und wie man damit in der Praxis umgehen soll

(wirklich nur stichwortartig, keine Romane!)